## Example: BX610 (High-z SFG) : Data Prep
### Prepare visibiity data from the calibrated MS restored by the local ALMA pipeline

    Split Calibrated Visibility Window by Window

    Perform Channel/Time averging

    Averging over continuum channel / spectral window partition

#### Load some essential Python modules + CASA 6 tasks/tools 

In [1]:

import sys,os,glob,io,socket
import logging
from pprint import pprint
import numpy as np

import casatasks as ctasks
import casatools as ctools

# Import wurlitzer for display real-time console logs
#   https://github.com/minrk/wurlitzer
%reload_ext wurlitzer

# for inline plots
%matplotlib inline
%config InlineBackend.figure_format = "retina"


Using the beta-version of CASA 6

In [2]:
print('casatools ver:',ctools.version_string())
print('casatasks ver:',ctasks.version_string())

casatools ver: 2019.172
casatasks ver: 2019.166


#### Import some convinient functions from `rxutils`

In [3]:
from rxutils.casa.proc import rmPointing        # help remove POINTING tables hidden under
from rxutils.casa.proc import setLogfile        # help reset the casa 6 log file
from rxutils.casa.proc import checkchflag       # help check channel-wise flagging stats
from rxutils.casa.proc import getcommonfreqs    # obatin the frequency coverage of one SPW in a specified frame (e.g. TOPO-LSRK)
from rxutils.casa.proc import rmColumns         # remove columns from MS

### general guideline

    Records with any flagged channels should be removed (from the Goodness evaluation)
    
    Auto-correlattion records (antenna='*&&&' where uvw=[0,0,0]) should be removed; at least set to flag=1 for tclean()
    
    To minimize the data file, try to bin in channel/time as much as possibile if the operation doesn't degrade the data signifcantly (smearing)
    
    If line only oacrupy a small fraction of a spectral window, try to averging the continuum portion if the line location is well known


### 2013.1.00059.S (Band 4, Cycle-2, Aravena M.) 

#### Spws Setup

+ spw=0  BB1  153GHz   CI 1-0
+ spw=1  BB2  155GHz   Continuum
+ spw=2  BB3  143GHz   CO 4-3
+ spw=3  BB4  141GHz   Continuum

Beam: 0.35"x0.30"

#### Export one MS per baseband

+ Combine the SPWs from each baseband into a single SPW
+ Transform the frame from TOPO to LSRK
+ Exclude the edge channels

Note: GMaKE requires the input MS contains only one SPW and one FIELD.

#### MS manupilation

+ The integration time is rebined to 60s (limited by the time-averging smearing)
+ The continuum SPW is averged into 1 representive channel (limited by the bandwidth smearing)
+ The point tables were removed[<sup>1</sup>](#fn1): they are only used for mosaic imaging, not in this specific case; for ALMA data, the table can be quite large even for channle-/time-averaged data)

<span id="fn1"> <sup>1</sup></span> We can't remove ms/POINTING table using `rmtables()` as its abence will crash certain CASA tasks/tools (e.g. listable); instead, we set it to have empty row, as it's done in `rmPointing()`.

In [4]:

# Switch working directory

demo_dir='/Users/Rui/Documents/Workspace/projects/GMaKE/examples/data/bx610/alma/2013.1.00059.S/'
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir(demo_dir)
setLogfile(demo_dir+'/'+'demo_bx610.log')

vis_name='/Volumes/D1/projects/hzdyn/2013.1.00059.S/science_goal.uid___A001_X12b_X239/group.uid___A001_X12b_X23a/member.uid___A001_X12b_X23b/calibrated/uid___A001_X12b_X23c_target.ms'
#ctasks.listobs(vis_name)


spw_list=['0','1','2','3']
bb_list=['1','2','3','4']
field='BX610'

# For BB1 & BB2 & BB3 & BB4: TOPO->LSKR

ctasks.flagdata(vis_name,autocorr=True,flagbackup=True)
for i in range(0,4):
    outvis='bb'+bb_list[i]+'.ms'
    os.system('rm -rf '+outvis)
    ctasks.mstransform(vis_name,outputvis=outvis,field='BX610',spw=spw_list[i],datacolumn='data',
                        regridms=True,outframe='lsrk',combinespws=True,mode='channel',start=1,nchan=478,width=1,
                        timeaverage=True,timebin='60s',maxuvwdistance=0.0,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)


# For BB1/BB3: three partitions
# For BB2/BB4: Channel-averging

bb_list=['1','3','2','4']
pb_list=['152.25GHz,153.00GHz,153.70GHz,154.80GHz',
         '142.25GHz,143.30GHz,143.80GHz,144.60GHz',
         '','']
chave_list=[[True,False,True],
            [True,False,True],
            [True],[True]]
         
for i in range(0,len(bb_list)):
    
    vis='bb'+bb_list[i]+'.ms'
    #rmColumns(vis,column='SIGMA_SPECTRUM') 
    #rmColumns(vis,column='WEIGHT_SPECTRUM')  
    for j in range(len(chave_list[i])):

        if  len(chave_list[i])==1:
            pt_name='.mfs'
            spw='0' 
        else:
            pt_name='.pt'+str(j+1)
            pb=pb_list[i].split(',')
            spw='0:'+pb[j]+'~'+pb[j+1]
        
        outvis=vis+pt_name
        print(vis,bb_list[i],spw,chave_list[i][j],outvis)
        os.system('rm -rf '+outvis)
        ctasks.mstransform(vis,outputvis=outvis,field='BX610',datacolumn='data',spw=spw,
                            chanaverage=chave_list[i][j],chanbin=2000,
                            timeaverage=False,keepflags=False,usewtspectrum=False)
        rmPointing(outvis)         


2019-11-18 17:30:40	INFO	flagdata::::casa	##########################################
2019-11-18 17:30:40	INFO	flagdata::::casa	##### Begin Task: flagdata           #####
2019-11-18 17:30:40	INFO	flagdata::::casa	flagdata( vis='/Volumes/D1/projects/hzdyn/2013.1.00059.S/science_goal.uid___A001_X12b_X239/group.uid___A001_X12b_X23a/member.uid___A001_X12b_X23b/calibrated/uid___A001_X12b_X23c_target.ms', mode='manual', autocorr=True, inpfile='', reason='any', tbuff=0.0, spw='', field='', antenna='', uvrange='', timerange='', correlation='', scan='', intent='', array='', observation='', feed='', clipminmax=[], datacolumn='DATA', clipoutside=True, channelavg=False, chanbin=1, timeavg=False, timebin='0s', clipzeros=False, quackinterval=1.0, quackmode='beg', quackincrement=False, tolerance=0.0, addantenna='', lowerlimit=0.0, upperlimit=90.0, ntime='scan', combinescans=False, timecutoff=4.0, freqcutoff=3.0, timefit='line', freqfit='poly', maxnpieces=7, flagdimension='freqtime', usewindowstats='no

bb1.ms 1 0:152.25GHz~153.00GHz True bb1.ms.pt1


2019-11-18 17:40:21	INFO	mstransform::::casa	##########################################
2019-11-18 17:40:21	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-11-18 17:40:21	INFO	mstransform::::casa	mstransform( vis='bb1.ms', outputvis='bb1.ms.pt1', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='0:152.25GHz~153.00GHz', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=False, chanaverage=True, chanbin=2000, hanning=False, regridms=False, mode='channel', nchan=-1, start=0, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='', veltype='radio', preaverage=False, timeaverage=False, timebin='0s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, want_cont=False, denoising_lib=True, nthreads=1, niter=1

bb1.ms 1 0:153.00GHz~153.70GHz False bb1.ms.pt2


2019-11-18 17:40:27	INFO	mstransform::::casa	Apply the transformations
2019-11-18 17:40:29	INFO	mstransform::::casa	Result mstransform: True
2019-11-18 17:40:29	INFO	mstransform::::casa	Task mstransform complete. Start time: 2019-11-18 11:40:25.223007 End time: 2019-11-18 11:40:28.518546
2019-11-18 17:40:29	INFO	mstransform::::casa	##### End Task: mstransform          #####
2019-11-18 17:40:29	INFO	mstransform::::casa	##########################################
2019-11-18 17:40:29	INFO	mstransform::::casa	##########################################
2019-11-18 17:40:29	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-11-18 17:40:29	INFO	mstransform::::casa	mstransform( vis='bb1.ms', outputvis='bb1.ms.pt3', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='0:153.70GHz~154.80GHz', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False,

...

2019-11-18 17:40:43	INFO	ParallelDataHelper::::casa	Apply the transformations
2019-11-18 17:40:46	INFO	mstransform::::casa	Result mstransform: True
2019-11-18 17:40:46	INFO	mstransform::::casa	Task mstransform complete. Start time: 2019-11-18 11:40:41.437634 End time: 2019-11-18 11:40:45.710853
2019-11-18 17:40:46	INFO	mstransform::::casa	##### End Task: mstransform          #####
2019-11-18 17:40:46	INFO	mstransform::::casa	##########################################
2019-11-18 17:40:46	INFO	mstransform::::casa	##########################################
2019-11-18 17:40:46	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-11-18 17:40:46	INFO	mstransform::::casa	mstransform( vis='bb2.ms', outputvis='bb2.ms.mfs', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='0', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=Fa

bb2.ms 2 0 True bb2.ms.mfs


2019-11-18 17:40:48	INFO	ParallelDataHelper::::casa	Apply the transformations
2019-11-18 17:40:51	INFO	mstransform::::casa	Result mstransform: True
2019-11-18 17:40:51	INFO	mstransform::::casa	Task mstransform complete. Start time: 2019-11-18 11:40:45.814868 End time: 2019-11-18 11:40:51.380754
2019-11-18 17:40:51	INFO	mstransform::::casa	##### End Task: mstransform          #####
2019-11-18 17:40:51	INFO	mstransform::::casa	##########################################
2019-11-18 17:40:51	INFO	mstransform::::casa	##########################################
2019-11-18 17:40:51	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-11-18 17:40:51	INFO	mstransform::::casa	mstransform( vis='bb4.ms', outputvis='bb4.ms.mfs', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='0', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=Fa

bb4.ms 4 0 True bb4.ms.mfs


2019-11-18 17:40:53	INFO	ParallelDataHelper::::casa	Apply the transformations
2019-11-18 17:40:57	INFO	mstransform::::casa	Result mstransform: True
2019-11-18 17:40:57	INFO	mstransform::::casa	Task mstransform complete. Start time: 2019-11-18 11:40:51.484037 End time: 2019-11-18 11:40:56.873622
2019-11-18 17:40:57	INFO	mstransform::::casa	##### End Task: mstransform          #####
2019-11-18 17:40:57	INFO	mstransform::::casa	##########################################


### 2015.1.00250.S (Band 6, Cycle-3, Aravena M.) 

#### Spws Setup

+ spw=0 BB1    250GHz   Continuum
+ spw=1 BB2    252GHz   CO 7-6 / CI 2-1
+ spw=2 BB3    234GHz   H2O
+ spw=3 BB4    236GHz   Continuum

Beam: 0.27"x0.24"

In [5]:
# Switch working directory

demo_dir='/Users/Rui/Documents/Workspace/projects/GMaKE/examples/data/bx610/alma/2015.1.00250.S/'
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir(demo_dir)
setLogfile(demo_dir+'/'+'demo_bx610.log')

vis_name='/Volumes/D1/projects/hzdyn/2015.1.00250.S/science_goal.uid___A001_X2fe_X20d/group.uid___A001_X2fe_X20e/member.uid___A001_X2fe_X20f/calibrated/uid___A001_X2fe_X20f_target.ms'
#ctasks.listobs(vis_name)


spw_list=['0','1','2','3']
bb_list=['1','2','3','4']
field='BX610'

# For BB1 & BB2 & BB3 & BB4: TOPO->LSKR
ctasks.flagdata(vis_name,autocorr=True,flagbackup=True)
for i in range(0,4):
    outvis='bb'+bb_list[i]+'.ms'
    os.system('rm -rf '+outvis)
    ctasks.mstransform(vis_name,outputvis=outvis,field='BX610',spw=spw_list[i],datacolumn='data',
                        regridms=True,outframe='lsrk',combinespws=True,mode='channel',start=1,nchan=238,width=1,
                        timeaverage=True,timebin='60s',maxuvwdistance=0.0,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)

    
# For BB2/BB3: three partitions
# For BB1/BB4: Channel-averging

bb_list=['1','4','2','3']
pb_list=['','',
         '250.50GHz,251.00GHz,251.50GHz,251.80GHz,252.25GHz,253.00GHz',
         '233.00GHz,233.90GHz,234.40GHz,235.50GHz']
chave_list=[[True],[True],[True,False,True,False,True],[True,False,True]]
         
for i in range(0,len(bb_list)):
    
    for j in range(len(chave_list[i])):

        if  len(chave_list[i])==1:
            pt_name='.mfs'
            spw='0' 
        else:
            pt_name='.pt'+str(j+1)
            pb=pb_list[i].split(',')
            spw='0:'+pb[j]+'~'+pb[j+1]
            
        vis='bb'+bb_list[i]+'.ms'
        outvis=vis+pt_name
        print(vis,bb_list[i],spw,chave_list[i][j],outvis)
        
        os.system('rm -rf '+outvis)
        ctasks.flagdata(vis,autocorr=True,flagbackup=False)
        ctasks.mstransform(vis,outputvis=outvis,field='BX610',datacolumn='data',spw=spw,
                            chanaverage=chave_list[i][j],chanbin=2000,
                            timeaverage=False,keepflags=False,usewtspectrum=False)
        rmPointing(outvis)


2019-11-18 18:13:28	INFO	flagdata::::casa	##########################################
2019-11-18 18:13:28	INFO	flagdata::::casa	##### Begin Task: flagdata           #####
2019-11-18 18:13:28	INFO	flagdata::::casa	flagdata( vis='/Volumes/D1/projects/hzdyn/2015.1.00250.S/science_goal.uid___A001_X2fe_X20d/group.uid___A001_X2fe_X20e/member.uid___A001_X2fe_X20f/calibrated/uid___A001_X2fe_X20f_target.ms', mode='manual', autocorr=True, inpfile='', reason='any', tbuff=0.0, spw='', field='', antenna='', uvrange='', timerange='', correlation='', scan='', intent='', array='', observation='', feed='', clipminmax=[], datacolumn='DATA', clipoutside=True, channelavg=False, chanbin=1, timeavg=False, timebin='0s', clipzeros=False, quackinterval=1.0, quackmode='beg', quackincrement=False, tolerance=0.0, addantenna='', lowerlimit=0.0, upperlimit=90.0, ntime='scan', combinescans=False, timecutoff=4.0, freqcutoff=3.0, timefit='line', freqfit='poly', maxnpieces=7, flagdimension='freqtime', usewindowstats='no

bb1.ms 1 0 True bb1.ms.mfs


2019-11-18 18:18:21	INFO	flagdata::::casa	##########################################
2019-11-18 18:18:21	INFO	flagdata::::casa	##### Begin Task: flagdata           #####
2019-11-18 18:18:21	INFO	flagdata::::casa	flagdata( vis='bb1.ms', mode='manual', autocorr=True, inpfile='', reason='any', tbuff=0.0, spw='', field='', antenna='', uvrange='', timerange='', correlation='', scan='', intent='', array='', observation='', feed='', clipminmax=[], datacolumn='DATA', clipoutside=True, channelavg=False, chanbin=1, timeavg=False, timebin='0s', clipzeros=False, quackinterval=1.0, quackmode='beg', quackincrement=False, tolerance=0.0, addantenna='', lowerlimit=0.0, upperlimit=90.0, ntime='scan', combinescans=False, timecutoff=4.0, freqcutoff=3.0, timefit='line', freqfit='poly', maxnpieces=7, flagdimension='freqtime', usewindowstats='none', halfwin=1, extendflags=True, winsize=3, timedev='', freqdev='', timedevscale=5.0, freqdevscale=5.0, spectralmax=1000000.0, spectralmin=0.0, antint_ref_antenna=''

bb4.ms 4 0 True bb4.ms.mfs


2019-11-18 18:18:26	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:26	INFO	FlagMSHandler::	Chunk = 2 [progress: 17%], Observation = 0, Array = 0, Scan = 12, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4921
2019-11-18 18:18:26	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:26	INFO	FlagMSHandler::	Chunk = 3 [progress: 26%], Observation = 0, Array = 0, Scan = 17, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4921
2019-11-18 18:18:26	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:26	INFO	FlagMSHandler::	Chunk = 4 [progress: 34%], Observation = 0, Array = 0, Scan = 19, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4921
2019-11-18 18:18:26	INFO	FlagMSHandler::	--------------

...

2019-11-18 18:18:52	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:52	INFO	FlagMSHandler::	Chunk = 1 [progress: 8%], Observation = 0, Array = 0, Scan = 10, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4662
2019-11-18 18:18:52	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:52	INFO	FlagMSHandler::	Chunk = 2 [progress: 17%], Observation = 0, Array = 0, Scan = 12, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4662
2019-11-18 18:18:52	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:52	INFO	FlagMSHandler::	Chunk = 3 [progress: 26%], Observation = 0, Array = 0, Scan = 17, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4662
2019-11-18 18:18:52	INFO	FlagMSHandler::	---------------

bb3.ms 3 0:233.90GHz~234.40GHz False bb3.ms.pt2


2019-11-18 18:18:56	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:56	INFO	FlagMSHandler::	Chunk = 4 [progress: 34%], Observation = 0, Array = 0, Scan = 19, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4662
2019-11-18 18:18:56	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:56	INFO	FlagMSHandler::	Chunk = 5 [progress: 43%], Observation = 0, Array = 0, Scan = 24, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4662
2019-11-18 18:18:56	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:56	INFO	FlagMSHandler::	Chunk = 6 [progress: 47%], Observation = 0, Array = 0, Scan = 26, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 1998
2019-11-18 18:18:56	INFO	FlagMSHandler::	--------------

bb3.ms 3 0:234.40GHz~235.50GHz True bb3.ms.pt3


2019-11-18 18:18:59	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:59	INFO	FlagMSHandler::	Chunk = 3 [progress: 26%], Observation = 0, Array = 0, Scan = 17, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4662
2019-11-18 18:18:59	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:59	INFO	FlagMSHandler::	Chunk = 4 [progress: 34%], Observation = 0, Array = 0, Scan = 19, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4662
2019-11-18 18:18:59	INFO	FlagMSHandler::	------------------------------------------------------------------------------------ 
2019-11-18 18:18:59	INFO	FlagMSHandler::	Chunk = 5 [progress: 43%], Observation = 0, Array = 0, Scan = 24, Field = 0 (BX610), Spw = 0, Channels = 238, Corrs = [ XX YY ], Total Rows = 4662
2019-11-18 18:18:59	INFO	FlagMSHandler::	--------------

### 2017.1.01045.S (Band 4, Cycle-5, Brisbin, L.) 

#### Spws Setup

+ spw=25/0 BB=1    143GHz   CO 4-3 
+ spw=27/1 BB=2    141GHz   Continuum
+ spw=29/2 BB=3    153GHz   CI 1-0
+ spw=31/3 BB=4    155GHz   Continuum

Beam: 0.02"x0.02"

#### Export one MS per baseband

+ Combine the SPWs from each baseband into a single SPW
+ Transform the frame from TOPO to LSRK
+ Exclude the edge channels

Note: GMaKE requires the input MS contains only one SPW and one FIELD.

#### MS manupilation

+ The integration time is rebined to 60s (limited by the time-averging smearing)
+ The continuum SPW is averged into 1 representive channel (limited by the bandwidth smearing)
+ The point tables were removed[<sup>1</sup>](#fn1): they are only used for mosaic imaging, not in this specific case; for ALMA data, the table can be quite large even for channle-/time-averaged data)
+ partion can reduce the data memory size to 1/3

<span id="fn1"> <sup>1</sup></span> We can't remove ms/POINTING table using `rmtables()` as its abence will crash certain CASA tasks/tools (e.g. listable); instead, we set it to have empty row, as it's done in `rmPointing()`.

In [ ]:

demo_dir='/Users/Rui/Documents/Workspace/projects/GMaKE/examples/data/bx610/alma/2017.1.01045.S/'
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir(demo_dir)
setLogfile(demo_dir+'/'+'demo_bx610.log')

repo='/Volumes/D1/proc/2017.1.01045.S/science_goal.uid___A001_X1288_X10c4/group.uid___A001_X1288_X10c5/member.uid___A001_X1288_X10c6/calibrated/'
uid_list=['uid___A002_Xc69057_X91a','uid___A002_Xc69057_Xc5c','uid___A002_Xc69057_X1293','uid___A002_Xc6a3db_X430','uid___A002_Xc6c0d5_X57ea']
spw_list=['25','27','29','31']
bb_list=['1','2','3','4']
field='BX610'

# Flag Auto-Coor

for uid in uid_list:
    ctasks.flagdata(repo+uid+'.ms',autocorr=True,flagbackup=False)

# For BB1 & BB2 & BB3 & BB4: TOPO->LSKR

for i in range(0,4):

    freqs,chanwidth=getcommonfreqs([repo+uid+'.ms' for uid in uid_list],
                                   [spw_list[i]]*len(uid_list),
                                   chanbin=4,frame='LSRK')
    # exclude one edge channel (defined in the binned frequency grid) from the mstransform() operation
    freqs=freqs[1:-1]
    print(min(freqs),max(freqs),chanwidth,len(freqs))
    
    vis_list=[]
    for uid in uid_list:
        
        vis_name=repo+uid+'.ms'
        outvis=uid+'_bb'+bb_list[i]+'.ms'
        os.system('rm -rf '+outvis)
        # use mode='freuqnecy' so if the basebands from different sessions are slightly offseted in TOPO, the concat result will still contain a single LSRK spw
        ctasks.mstransform(vis_name,outputvis=outvis,field='BX610',spw=spw_list[i],datacolumn='corrected',
                            regridms=True,outframe='lsrk',combinespws=True,mode='frequency',\
                            start=str(min(freqs))+'Hz',nchan=len(freqs),width=str(chanwidth)+'Hz',
                            timeaverage=True,timebin='60s',maxuvwdistance=0.0,
                            keepflags=False,usewtspectrum=False,intent='OBSERVE_TARGET#ON_SOURCE')          
        rmPointing(outvis,verbose=True)
        vis_list+=[outvis]

    os.system('rm -rf '+'bb'+bb_list[i]+'.ms')
    ctasks.concat(vis=vis_list,concatvis='bb'+bb_list[i]+'.ms',copypointing=True)
    ctasks.listobs('bb'+bb_list[i]+'.ms')
    checkchflag('bb'+bb_list[i]+'.ms')

    
# For BB1/BB3: three partitions
# For BB2/BB4: Channel-averging

bb_list=['1','3','2','4']
pb_list=['142.25GHz,143.30GHz,143.80GHz,144.80GHz',
         '152.25GHz,153.00GHz,153.70GHz,154.75GHz',
         '','']
chave_list=[[True,False,True],
            [True,False,True],
            [True],[True]]
         
for i in range(0,len(bb_list)):     
    for j in range(len(chave_list[i])):

        if  len(chave_list[i])==1:
            pt_name='.mfs'
            spw='0' 
        else:
            pt_name='.pt'+str(j+1)
            pb=pb_list[i].split(',')
            spw='0:'+pb[j]+'~'+pb[j+1]
            
        vis='bb'+bb_list[i]+'.ms'
        outvis=vis+pt_name
        print(vis,bb_list[i],spw,chave_list[i][j],outvis)
        
        os.system('rm -rf '+outvis)
        
        ctasks.mstransform(vis,outputvis=outvis,field='BX610',datacolumn='data',spw=spw,
                            chanaverage=chave_list[i][j],chanbin=2000,
                            timeaverage=False,keepflags=False,usewtspectrum=False)
        rmPointing(outvis)    


2019-11-18 18:40:46	INFO	flagdata::::casa	##########################################
2019-11-18 18:40:46	INFO	flagdata::::casa	##### Begin Task: flagdata           #####
2019-11-18 18:40:46	INFO	flagdata::::casa	flagdata( vis='/Volumes/D1/proc/2017.1.01045.S/science_goal.uid___A001_X1288_X10c4/group.uid___A001_X1288_X10c5/member.uid___A001_X1288_X10c6/calibrated/uid___A002_Xc69057_X91a.ms', mode='manual', autocorr=True, inpfile='', reason='any', tbuff=0.0, spw='', field='', antenna='', uvrange='', timerange='', correlation='', scan='', intent='', array='', observation='', feed='', clipminmax=[], datacolumn='DATA', clipoutside=True, channelavg=False, chanbin=1, timeavg=False, timebin='0s', clipzeros=False, quackinterval=1.0, quackmode='beg', quackincrement=False, tolerance=0.0, addantenna='', lowerlimit=0.0, upperlimit=90.0, ntime='scan', combinescans=False, timecutoff=4.0, freqcutoff=3.0, timefit='line', freqfit='poly', maxnpieces=7, flagdimension='freqtime', usewindowstats='none', hal

142636922541.6709 144496401127.33594 3906467.6169433594 477


2019-11-18 19:07:50	INFO	mstransform::::casa	Combine spws 25 into new output spw
2019-11-18 19:07:50	INFO	mstransform::::casa	Parse regridding parameters
2019-11-18 19:07:50	INFO	ParallelDataHelper::::casa	Parse time averaging parameters
2019-11-18 19:07:50	INFO	MSTransformManager::parseMsSpecParams	Input file name is /Volumes/D1/proc/2017.1.01045.S/science_goal.uid___A001_X1288_X10c4/group.uid___A001_X1288_X10c5/member.uid___A001_X1288_X10c6/calibrated/uid___A002_Xc69057_X91a.ms
2019-11-18 19:07:50	INFO	MSTransformManager::parseMsSpecParams	Data column is CORRECTED
2019-11-18 19:07:50	INFO	MSTransformManager::parseMsSpecParams	Output file name is uid___A002_Xc69057_X91a_bb1.ms
2019-11-18 19:07:50	INFO	MSTransformManager::parseMsSpecParams	Re-index is enabled 
2019-11-18 19:07:50	INFO	MSTransformManager::parseMsSpecParams	Tile shape is [0]
2019-11-18 19:07:50	INFO	MSTransformManager::parseDataSelParams	field selection is BX610
2019-11-18 19:07:50	INFO	MSTransformManager::parseDataSelPa

Before rmPointing()
1.2G	uid___A002_Xc69057_X91a_bb1.ms
After rmPointing()
418M	uid___A002_Xc69057_X91a_bb1.ms


2019-11-18 19:16:37	INFO	mstransform::::casa	##########################################
2019-11-18 19:16:37	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-11-18 19:16:37	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/proc/2017.1.01045.S/science_goal.uid___A001_X1288_X10c4/group.uid___A001_X1288_X10c5/member.uid___A001_X1288_X10c6/calibrated/uid___A002_Xc69057_Xc5c.ms', outputvis='uid___A002_Xc69057_Xc5c_bb1.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='25', scan='', antenna='', correlation='', timerange='', intent='OBSERVE_TARGET#ON_SOURCE', array='', uvrange='', observation='', feed='', datacolumn='corrected', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='frequency', nchan=477, start='142636922541.6709Hz', width='3906467.6169433594Hz', nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='l